<a href="https://colab.research.google.com/github/prikshit-2000/IMDB-movie_review-sentimental-analysis/blob/master/IMDB_Sentimental_Analysis_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [3]:
import tensorflow_datasets as tfds
imdb , info = tfds.load('imdb_reviews',with_info=True,as_supervised=True)


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteFZLVBS/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteFZLVBS/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteFZLVBS/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [53]:
import numpy as np

In [54]:
train_data,test_data = imdb['train'],imdb['test']

In [65]:

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
  training_sentences.append(s.numpy().decode('utf8'))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())
  
# training_labels_final = np.array(training_labels)
# testing_labels_final = np.array(testing_labels)

In [94]:
print(testing_sentences[10:11])

['I\'ll give it a two because it has a lot of music, otherwise it would be a one.<br /><br />I saw this movie for the first time tonight and it\'s the first "Road" picture I\'ve seen. I was expecting waaaaay better. Robert Osborn says this is the best of the Road movies. If that\'s true I needn\'t bother to see the others. The best thing about this movie is that it has a lot of songs in the first half, but that\'s balanced out by only one production number with dancing in the entire movie.<br /><br />I didn\'t like the movie. Neither Hope nor Crosby came across all that well, their characters weren\'t very charming, the movie was not funny at all, most of the dialog was just lame filler, there wasn\'t much action, there wasn\'t much spectacle.<br /><br />The movie wasn\'t what I expected. I was expecting more "Road," but there isn\'t much. They quickly make it to the palace and then most of the movie takes place there, until the end. I was also expecting a lot more of the famous "road"

In [67]:
print(testing_labels[0:10])

[1, 1, 0, 0, 1, 1, 1, 1, 0, 1]


In [68]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)
print(testing_labels_final[0:10])

[1 1 0 0 1 1 1 1 0 1]


In [69]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
oov_tok = '<OOV>'
trunc_type = 'post'


In [70]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [71]:
tokenizer = Tokenizer(num_words=vocab_size,oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length,truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)


In [72]:
model = tf.keras.Sequential([
                            tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length=max_length),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(6,activation='relu'),
                             tf.keras.layers.Dense(1,activation='sigmoid')
])

In [73]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_5 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 6)                 11526     
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [74]:
num_epochs = 10
model.fit(padded,training_labels_final,epochs=num_epochs,validation_data=(testing_padded,testing_labels_final))


Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 0.4984 - accuracy: 0.7400 - val_loss: 0.3430 - val_accuracy: 0.8521
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2436 - accuracy: 0.9056 - val_loss: 0.3553 - val_accuracy: 0.8456
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.1039 - accuracy: 0.9720 - val_loss: 0.4294 - val_accuracy: 0.8319
Epoch 4/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0305 - accuracy: 0.9956 - val_loss: 0.5355 - val_accuracy: 0.8268
Epoch 5/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0087 - accuracy: 0.9990 - val_loss: 0.6065 - val_accuracy: 0.8244
Epoch 6/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0033 - accuracy: 0.9998 - val_loss: 0.6373 - val_accuracy: 0.8286
Epoch 7/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0015 - accuracy: 0.9999 - val_loss: 0.6884 - val_accuracy: 0.8274
Epoch 

In [75]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)# vocab_size,embedding_dim


(10000, 16)


In [78]:
reverse_word_index = dict([(value,key) for key,value in  word_index.items()])
def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print(training_sentences[3])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received
This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.


In [77]:
reverse_word_index

{1: '<OOV>',
 2: 'the',
 3: 'and',
 4: 'a',
 5: 'of',
 6: 'to',
 7: 'is',
 8: 'br',
 9: 'in',
 10: 'it',
 11: 'i',
 12: 'this',
 13: 'that',
 14: 'was',
 15: 'as',
 16: 'for',
 17: 'with',
 18: 'movie',
 19: 'but',
 20: 'film',
 21: 'on',
 22: 'not',
 23: 'you',
 24: 'are',
 25: 'his',
 26: 'have',
 27: 'he',
 28: 'be',
 29: 'one',
 30: 'all',
 31: 'at',
 32: 'by',
 33: 'an',
 34: 'they',
 35: 'who',
 36: 'so',
 37: 'from',
 38: 'like',
 39: 'her',
 40: 'or',
 41: 'just',
 42: 'about',
 43: "it's",
 44: 'out',
 45: 'if',
 46: 'has',
 47: 'some',
 48: 'there',
 49: 'what',
 50: 'good',
 51: 'more',
 52: 'when',
 53: 'very',
 54: 'up',
 55: 'no',
 56: 'time',
 57: 'she',
 58: 'even',
 59: 'my',
 60: 'would',
 61: 'which',
 62: 'only',
 63: 'story',
 64: 'really',
 65: 'see',
 66: 'their',
 67: 'had',
 68: 'can',
 69: 'were',
 70: 'me',
 71: 'well',
 72: 'than',
 73: 'we',
 74: 'much',
 75: 'been',
 76: 'bad',
 77: 'get',
 78: 'will',
 79: 'do',
 80: 'also',
 81: 'into',
 82: 'people',
 8

In [91]:
predict_review ='This movie is great and mindblowing'
predict_review = tokenizer.texts_to_sequences([predict_review])
# print(predict_review)
padded_predict_review = pad_sequences(predict_review,maxlen=max_length,truncating=trunc_type)
# print(padded_predict_review)
pred=model.predict(padded_predict_review)
pred_review=(np.where(pred>=0.5,1,0))
if pred_review ==1:
  print("This movie has a good rating")
else:
  print("This movie has a low rating")

This movie has a good rating


In [93]:
predict_review ='This movie is bad and boring'
predict_review = tokenizer.texts_to_sequences([predict_review])
# print(predict_review)
padded_predict_review = pad_sequences(predict_review,maxlen=max_length,truncating=trunc_type)
# print(padded_predict_review)
pred=model.predict(padded_predict_review)
pred_review=(np.where(pred>=0.5,1,0))
if pred_review ==1:
  print("This movie has a good rating")
else:
  print("This movie has a low rating")

This movie has a low rating
